# 🤖 LLM Post-Filter Experiment: GLITCH+LLM Pipeline

**Focus**: Evaluate **GLITCH + LLM** hybrid approach vs **GLITCH-only** baseline.

## 🔬 Experiment Pipeline:

1. **Data Preparation**: GLITCH detections + context extracted *(01_data_extraction.py)*
2. **LLM Filtering**: Apply GPT-4o mini post-filtering  
3. **Performance Evaluation**: Calculate precision/recall improvements

## 🎯 Expected Outcomes:
- **Precision**: 50-300% improvement
- **Recall**: >90% retention  
- **FP Reduction**: Significant decrease in false alarms


## 🔧 Setup and Configuration

**Required**: Set OpenAI API key: `export OPENAI_API_KEY="your-key"`

**Model**: GPT-4o mini for cost-effective evaluation


In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import warnings
warnings.filterwarnings('ignore')

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Add llm-postfilter modules to path
project_root = Path.cwd().parent.parent.parent
sys.path.append(str(project_root / "src"))

# Import llm-postfilter pipeline components
from llm_postfilter import (
    GLITCHLLMFilter, 
    HybridEvaluator,
    SecuritySmellPrompts,
    SecuritySmell
)

print(f"🏠 Project root: {project_root}")
print(f"📍 Working directory: {Path.cwd()}")

# Check OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ OpenAI API key found (length: {len(api_key)})")
else:
    print("❌ OpenAI API key not found - set OPENAI_API_KEY environment variable")

print("🚀 LLM Post-Filter Experiment Ready!")

🏠 Project root: /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval
📍 Working directory: /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm_postfilter/notebooks
✅ OpenAI API key found (length: 164)
🚀 LLM Post-Filter Experiment Ready!


## 📁 Step 1: Load Context-Enhanced Data

**Load detection files with code context prepared by 01_data_extraction.py**


In [2]:
# Setup directories
data_dir = project_root / "experiments/llm_postfilter/data"
context_dir = data_dir / "with_context"

# Find context-enhanced files
context_enhanced_files = list(context_dir.glob("*_with_context.csv"))

if context_enhanced_files:
    print(f"📁 Found {len(context_enhanced_files)} context-enhanced files:")
    for file in context_enhanced_files:
        df = pd.read_csv(file)
        tp_count = df['is_true_positive'].sum()
        fp_count = len(df) - tp_count
        context_success = df['context_success'].sum()
        print(f"  📄 {file.name}: {len(df)} detections ({tp_count} TP, {fp_count} FP, {context_success} with context)")
    
    print(f"\n✅ Context-enhanced data ready for LLM analysis")
    
else:
    print("❌ No context-enhanced files found!")
    print("➡️  Run 01_data_extraction.py first to prepare the data")

📁 Found 6 context-enhanced files:
  📄 puppet_suspicious_comment_detections_with_context.csv: 23 detections (9 TP, 14 FP, 23 with context)
  📄 puppet_use_of_weak_cryptography_algorithms_detections_with_context.csv: 7 detections (4 TP, 3 FP, 7 with context)
  📄 puppet_hard_coded_secret_detections_with_context.csv: 66 detections (9 TP, 57 FP, 66 with context)
  📄 chef_use_of_weak_cryptography_algorithms_detections_with_context.csv: 2 detections (1 TP, 1 FP, 2 with context)
  📄 chef_hard_coded_secret_detections_with_context.csv: 46 detections (9 TP, 37 FP, 46 with context)
  📄 chef_suspicious_comment_detections_with_context.csv: 10 detections (4 TP, 6 FP, 10 with context)

✅ Context-enhanced data ready for LLM analysis


## 📝 Step 2: Review LLM Prompt Design

Review the formal security smell definitions used for LLM evaluation.


In [3]:
# Display formal definitions for each security smell
print("📝 Security Smell Definitions for LLM")
print("=" * 40)

for smell in SecuritySmell:
    definition = SecuritySmellPrompts.DEFINITIONS[smell]
    lines = definition.strip().split('\n')[:3]
    print(f"\n📌 {smell.value}")
    for line in lines:
        print(f"  {line}")
    print(f"  ... ({len(definition.split())} words total)")

print(f"\n✅ {len(SecuritySmell)} smell categories with formal definitions ready")

📝 Security Smell Definitions for LLM

📌 Hard-coded secret
  A hard-coded secret is a security vulnerability where sensitive information such as passwords, API keys, tokens, certificates, or other credentials are directly embedded in the source code as literal strings or variables, rather than being securely stored and retrieved from external configuration systems, environment variables, or secret management services.
  
  Key characteristics:
  ... (128 words total)

📌 Suspicious comment
  A suspicious comment is a code comment that indicates potential security issues, incomplete security implementations, or areas requiring security attention. These comments often signal unfinished work, security bypasses, or acknowledged vulnerabilities that may pose risks.
  
  Key characteristics:
  ... (119 words total)

📌 Use of weak cryptography algorithms
  Use of weak cryptography algorithms refers to the implementation or configuration of cryptographic functions that are known to be vulnerable

## 🔧 Step 3: Initialize LLM Pipeline

Setup GLITCH+LLM hybrid detection pipeline with GPT-4o mini.


In [4]:
# Initialize the LLM filter pipeline
if api_key:
    print("🔧 Initializing GLITCH+LLM pipeline...")
    
    # Create components
    llm_filter = GLITCHLLMFilter(
        project_root=project_root,
        api_key=api_key,
        model="gpt-4o-mini"
    )
    evaluator = HybridEvaluator(project_root)
    
    # Setup directories
    results_dir = data_dir / "llm_results"
    results_dir.mkdir(exist_ok=True)
    
    print("✅ Pipeline ready:")
    print(f"  🤖 Model: {llm_filter.llm_client.model}")
    print(f"  📊 Results → {results_dir}")
    
else:
    print("❌ Pipeline initialization failed - API key required")
    print("Set OPENAI_API_KEY and restart kernel")

2025-08-06 15:55:10,062 - llm_postfilter.llm_client - INFO - Initialized GPT-4o mini client with model: gpt-4o-mini
2025-08-06 15:55:10,062 - llm_postfilter.llm_filter - INFO - Initialized GLITCH+LLM filter pipeline


🔧 Initializing GLITCH+LLM pipeline...
✅ Pipeline ready:
  🤖 Model: gpt-4o-mini
  📊 Results → /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm_postfilter/data/llm_results


## 🚀 Step 4: Run LLM Post-Filtering

Apply LLM post-filtering to GLITCH detections and measure improvements.


In [5]:
if api_key and 'context_enhanced_files' in locals():
    print(f"🔍 Processing {len(context_enhanced_files)} context-enhanced files:")
    for file in context_enhanced_files:
        df = pd.read_csv(file)
        tp_count = df['is_true_positive'].sum()
        fp_count = len(df) - tp_count
        context_success = df['context_success'].sum()
        print(f"  📁 {file.name}: {len(df)} detections ({tp_count} TP, {fp_count} FP)")
    
    print(f"\n🚀 Starting LLM post-filtering...")
    
    # Process each context-enhanced file
    filtered_results = {}
    
    for i, context_file in enumerate(context_enhanced_files):
        print(f"\n🔄 Processing {i+1}/{len(context_enhanced_files)}: {context_file.name}")
        
        try:
            # Run LLM filtering
            filtered_df = llm_filter.filter_detections(context_file, results_dir)
            filtered_results[context_file.stem] = filtered_df
            
            # Summary stats
            total = len(filtered_df)
            kept = filtered_df['keep_detection'].sum()
            original_tp = filtered_df['is_true_positive'].sum() 
            kept_tp = filtered_df[filtered_df['keep_detection']]['is_true_positive'].sum()
            
            print(f"✅ Kept {kept}/{total} ({kept/total:.1%}) | TP retention: {kept_tp}/{original_tp} ({kept_tp/original_tp:.1%})")
            
        except Exception as e:
            print(f"❌ Error: {e}")
            logger.error(f"Failed to process {context_file}: {e}")
    
    print(f"\n🎉 LLM filtering completed! Results → {results_dir}")
    
elif not api_key:
    print("❌ Skipping - API key required")
else:
    print("❌ Skipping - no context files (run context extraction first)")

2025-08-06 00:39:42,422 - hybrid.llm_filter - INFO - Starting LLM post-filtering pipeline for puppet_suspicious_comment_detections_with_context.csv
2025-08-06 00:39:42,423 - hybrid.llm_filter - INFO - Loaded 23 detections from puppet_suspicious_comment_detections_with_context.csv
2025-08-06 00:39:42,423 - hybrid.llm_filter - INFO - Extracting code context for detections...
2025-08-06 00:39:42,430 - hybrid.context_extractor - INFO - Successfully extracted context for 23/23 detections
2025-08-06 00:39:42,430 - hybrid.llm_filter - INFO - Context extraction: 23/23 successful (100.0%)
2025-08-06 00:39:42,431 - hybrid.llm_filter - INFO - Generated 23 prompts for LLM evaluation
2025-08-06 00:39:42,431 - hybrid.llm_filter - INFO - Starting LLM evaluation of 23 detections...
2025-08-06 00:39:42,431 - hybrid.llm_client - INFO - Starting batch evaluation of 23 prompts


🔍 Processing 6 context-enhanced files:
  📁 puppet_suspicious_comment_detections_with_context.csv: 23 detections (9 TP, 14 FP)
  📁 puppet_use_of_weak_cryptography_algorithms_detections_with_context.csv: 7 detections (4 TP, 3 FP)
  📁 puppet_hard_coded_secret_detections_with_context.csv: 66 detections (9 TP, 57 FP)
  📁 chef_use_of_weak_cryptography_algorithms_detections_with_context.csv: 2 detections (1 TP, 1 FP)
  📁 chef_hard_coded_secret_detections_with_context.csv: 46 detections (9 TP, 37 FP)
  📁 chef_suspicious_comment_detections_with_context.csv: 10 detections (4 TP, 6 FP)

🚀 Starting LLM post-filtering...

🔄 Processing 1/6: puppet_suspicious_comment_detections_with_context.csv


2025-08-06 00:39:42,998 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:39:44,533 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:39:45,307 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:39:46,648 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:39:47,170 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:39:47,172 - hybrid.llm_filter - INFO - LLM progress: 5/23 (21.7%)
2025-08-06 00:39:48,277 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:39:49,113 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:39:50,302 - httpx - INFO - HTTP Request: POST https://api.o

✅ Kept 3/23 (13.0%) | TP retention: 3/9 (33.3%)

🔄 Processing 2/6: puppet_use_of_weak_cryptography_algorithms_detections_with_context.csv


2025-08-06 00:40:06,449 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:40:07,110 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:40:08,188 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:40:09,185 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:40:10,473 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:40:10,508 - hybrid.llm_filter - INFO - LLM progress: 5/7 (71.4%)
2025-08-06 00:40:11,147 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:40:12,869 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:40:12,872 - hybrid.llm_filter - INFO - LLM progress: 7/7 (10

...

✅ Kept 1/2 (50.0%) | TP retention: 1/1 (100.0%)

🔄 Processing 5/6: chef_hard_coded_secret_detections_with_context.csv


2025-08-06 00:41:30,965 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:41:32,079 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:41:33,061 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:41:33,973 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:41:35,560 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:41:35,565 - hybrid.llm_filter - INFO - LLM progress: 5/46 (10.9%)
2025-08-06 00:41:36,106 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:41:37,161 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:41:38,070 - httpx - INFO - HTTP Request: POST https://api.o

✅ Kept 5/46 (10.9%) | TP retention: 3/9 (33.3%)

🔄 Processing 6/6: chef_suspicious_comment_detections_with_context.csv


2025-08-06 00:42:18,402 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:42:18,900 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:42:19,819 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:42:20,848 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:42:22,034 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:42:22,037 - hybrid.llm_filter - INFO - LLM progress: 5/10 (50.0%)
2025-08-06 00:42:22,863 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:42:24,113 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 00:42:25,003 - httpx - INFO - HTTP Request: POST https://api.o

✅ Kept 1/10 (10.0%) | TP retention: 1/4 (25.0%)

🎉 LLM filtering completed! Results → /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm-postfilter/data/llm_results


## 📈 Step 5: Evaluate Performance Improvement

Calculate precision, recall, and F1 improvements from LLM post-filtering.


In [6]:
if api_key and 'filtered_results' in locals():
    print("📊 Evaluating GLITCH vs GLITCH+LLM Performance")
    print("=" * 50)
    
    # Organize results by IaC tool
    evaluation_results = {}
    
    for tool in ['chef', 'puppet']:
        tool_filtered_dfs = []
        for key, filtered_df in filtered_results.items():
            if key.startswith(tool):
                tool_filtered_dfs.append(filtered_df)
        
        if tool_filtered_dfs:
            tool_results = evaluator.evaluate_iac_tool(tool_filtered_dfs, tool.title())
            evaluation_results[tool] = tool_results
    
    # Save evaluation results
    evaluation_dir = results_dir / "evaluation"
    evaluation_dir.mkdir(exist_ok=True)
    
    if evaluation_results:
        summary_df = evaluator.save_evaluation_results(evaluation_results, evaluation_dir)
        
        print("\n🎯 EXPERIMENT RESULTS")
        print("=" * 40)
        
        # Display key findings
        for _, row in summary_df.iterrows():
            tool = row['IaC_Tool']
            smell = row['Security_Smell']
            baseline_precision = row['Baseline_Precision']
            llm_precision = row['LLM_Precision']
            precision_improvement = row['Precision_Improvement']
            fp_reduction = row['FP_Reduction']
            tp_retention = row['TP_Retention']
            
            print(f"\n📌 {tool} - {smell}:")
            print(f"  Precision: {baseline_precision:.3f} → {llm_precision:.3f} ({precision_improvement:+.1%})")
            print(f"  FP↓: {fp_reduction:.1%} | TP retained: {tp_retention:.1%}")
        
        # Overall improvements
        avg_precision_improvement = summary_df['Precision_Improvement'].mean()
        avg_fp_reduction = summary_df['FP_Reduction'].mean()
        avg_tp_retention = summary_df['TP_Retention'].mean()
        
        print(f"\n🚀 OVERALL OUTCOMES:")
        print(f"  📈 Precision improvement: {avg_precision_improvement:+.1%}")
        print(f"  📉 FP reduction: {avg_fp_reduction:.1%}")
        print(f"  🎯 TP retention: {avg_tp_retention:.1%}")
        
        print(f"\n💾 Detailed results → {evaluation_dir}")
        
    else:
        print("❌ No evaluation results available")
        
else:
    print("⏭️ Skipping evaluation - run LLM filtering first")

2025-08-06 00:42:27,084 - hybrid.evaluator - INFO - Saved detailed results to /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm-postfilter/data/llm_results/evaluation/hybrid_evaluation_results.json
2025-08-06 00:42:27,086 - hybrid.evaluator - INFO - Saved summary table to /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm-postfilter/data/llm_results/evaluation/hybrid_evaluation_summary.csv
2025-08-06 00:42:27,088 - hybrid.evaluator - INFO - Saved performance comparison to /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm-postfilter/data/llm_results/evaluation/performance_comparison_table.csv


📊 Evaluating GLITCH vs GLITCH+LLM Performance

🎯 EXPERIMENT RESULTS

📌 chef - Use of weak cryptography algorithms:
  Precision: 0.500 → 1.000 (+100.0%)
  FP↓: 100.0% | TP retained: 100.0%

📌 chef - Hard-coded secret:
  Precision: 0.196 → 0.600 (+206.7%)
  FP↓: 94.6% | TP retained: 33.3%

📌 chef - Suspicious comment:
  Precision: 0.400 → 1.000 (+150.0%)
  FP↓: 100.0% | TP retained: 25.0%

📌 puppet - Suspicious comment:
  Precision: 0.391 → 1.000 (+155.6%)
  FP↓: 100.0% | TP retained: 33.3%

📌 puppet - Use of weak cryptography algorithms:
  Precision: 0.571 → 0.600 (+5.0%)
  FP↓: 33.3% | TP retained: 75.0%

📌 puppet - Hard-coded secret:
  Precision: 0.136 → 0.643 (+371.4%)
  FP↓: 91.2% | TP retained: 100.0%

🚀 OVERALL OUTCOMES:
  📈 Precision improvement: +164.8%
  📉 FP reduction: 86.5%
  🎯 TP retention: 61.1%

💾 Detailed results → /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm-postfilter/data/l

## 📁 Generated Files & Transparency

Complete experimental transparency through intermediate files.


In [7]:
print("📁 Generated Files & Transparency")
print("=" * 40)

print("\n🔍 Context Files (LLM input):")
if 'context_dir' in locals():
    context_files = list(context_dir.glob("*.csv"))
    for file in context_files:
        size_kb = file.stat().st_size // 1024
        print(f"  📄 {file.name} ({size_kb} KB)")
    print(f"  📁 {context_dir}")
else:
    print("  ❌ No context files generated")

print("\n🤖 LLM Results:")
if 'results_dir' in locals() and results_dir.exists():
    result_files = list(results_dir.glob("*.csv")) + list(results_dir.glob("*.json"))
    for file in result_files:
        size_kb = file.stat().st_size // 1024
        if file.name.endswith("_prompts_and_responses.json"):
            print(f"  📝 {file.name} ({size_kb} KB) - Full prompts & LLM responses")
        elif file.name.endswith("_llm_filtered.csv"):
            print(f"  📊 {file.name} ({size_kb} KB) - Filtered detections")
        else:
            print(f"  📄 {file.name} ({size_kb} KB)")
    print(f"  📁 {results_dir}")
else:
    print("  ❌ No LLM results generated")

print("\n📊 Evaluation:")
if 'evaluation_dir' in locals() and evaluation_dir.exists():
    eval_files = list(evaluation_dir.glob("*.csv")) + list(evaluation_dir.glob("*.json"))
    for file in eval_files:
        size_kb = file.stat().st_size // 1024
        print(f"  📄 {file.name} ({size_kb} KB)")
    print(f"  📁 {evaluation_dir}")
else:
    print("  ❌ No evaluation results generated")

print("\n💡 Full transparency: code snippets, prompts, LLM decisions, metrics")

📁 Generated Files & Transparency

🔍 Context Files (LLM input):
  📄 puppet_suspicious_comment_detections_with_context.csv (16 KB)
  📄 puppet_use_of_weak_cryptography_algorithms_detections_with_context.csv (5 KB)
  📄 puppet_hard_coded_secret_detections_with_context.csv (47 KB)
  📄 chef_use_of_weak_cryptography_algorithms_detections_with_context.csv (1 KB)
  📄 chef_hard_coded_secret_detections_with_context.csv (37 KB)
  📄 chef_suspicious_comment_detections_with_context.csv (7 KB)
  📁 /Users/colemei/Library/Mobile Documents/com~apple~CloudDocs/01.Work/04.Master/Course/Research Program/Project/LLM-IaC-SecEval/experiments/llm-postfilter/data/detections/with_context

🤖 LLM Results:
  📊 chef_hard_coded_secret_detections_with_context_llm_filtered.csv (39 KB) - Filtered detections
  📊 chef_use_of_weak_cryptography_algorithms_detections_with_context_llm_filtered.csv (1 KB) - Filtered detections
  📊 puppet_suspicious_comment_detections_with_context_llm_filtered.csv (17 KB) - Filtered detections
  